In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/29 16:17:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/29 16:17:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# Read the csv files from datasets

#Fake news
df_fake = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/Fake.csv")
df_fake.show()

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|
| Papa John’s Foun...|A centerpiece of ...|   News|December 21, 2017|
| WATCH: Paul Ryan..

In [3]:
#Truth news
df_true = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/True.csv")
df_true.show()

+--------------------+--------------------+--------------------+------------------+
|               title|                text|             subject|              date|
+--------------------+--------------------+--------------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Factbox: Trump on...|The following sta...|        politicsNews|December 29, 2017 |
|Trump on Twitter ...|The following sta...|        politicsNews|December 29,

In [4]:
# count of data and the. umber of coloumns
print((df_fake.count(), len(df_fake.columns)))
print((df_true.count(), len(df_true.columns)))

(23489, 4)
(21417, 4)


In [5]:
#Add the column for fake news and also a column for true news
from pyspark.sql.functions import lit

df_fake = df_fake.withColumn("Flag", lit("Fake_News"))
df_fake.show()

+--------------------+--------------------+-------+-----------------+---------+
|               title|                text|subject|             date|     Flag|
+--------------------+--------------------+-------+-----------------+---------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_News|
| WATCH: Brand-New...|Just when you mig.

In [6]:
df_true = df_true.withColumn("Flag", lit("True_News"))
df_true.show()

+--------------------+--------------------+--------------------+------------------+---------+
|               title|                text|             subject|              date|     Flag|
+--------------------+--------------------+--------------------+------------------+---------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |True_News|
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |True_News|
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |True_News|
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Factbox: Trump on...|The following sta...|        politicsN

In [7]:
#concat the two tables
df_concat = df_fake.unionByName(df_true, allowMissingColumns=True)
df_concat.show()

+--------------------+--------------------+-------+-----------------+---------+
|               title|                text|subject|             date|     Flag|
+--------------------+--------------------+-------+-----------------+---------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_News|
| WATCH: Brand-New...|Just when you mig.

In [8]:
#specifying the row number to each row

from pyspark.sql.functions import monotonically_increasing_id 

df_concat = df_concat.select("*").withColumn("id", monotonically_increasing_id())

df_concat.show()

+--------------------+--------------------+-------+-----------------+---------+---+
|               title|                text|subject|             date|     Flag| id|
+--------------------+--------------------+-------+-----------------+---------+---+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|  0|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|  1|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|  2|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|  3|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|  4|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|  5|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|  6|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|  7|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_Ne

In [9]:
print((df_concat.count(), len(df_concat.columns)))

(44906, 6)


In [10]:
#### has to randomly shuffle the data
import pyspark.sql.functions as F

df_concat = df_concat.select("*").orderBy(F.rand())
df_concat.show()

+--------------------+--------------------+----------------+-------------------+---------+------------+
|               title|                text|         subject|               date|     Flag|          id|
+--------------------+--------------------+----------------+-------------------+---------+------------+
|“HARRY, HARRY!” C...|President Trump s...| Government News|        Jul 2, 2017|Fake_News| 34359741452|
| Conservatives Ar...|If it weren t for...|            News|  December 31, 2016|Fake_News|  8589934724|
|ERIC TRUMP School...|                    |       left-news|       Oct 23, 2016|Fake_News| 42949675948|
|MUSLIM TEENS Stag...|"Oh and as an add...| fjs);}(document|           'script'|Fake_News| 51539608638|
|Classless Obama C...|Lack of leadershi...|        politics|        Dec 3, 2017|Fake_News| 25769803897|
|Republican Trump'...|WASHINGTON (Reute...|    politicsNews|       May 5, 2016 |True_News| 94489282035|
|WHO NEEDS DEMOCRA...|No wonder we ve h...|       left-news|    

In [11]:
# Remove the unwanted coloumn

df_concat =  df_concat.drop(df_concat["date"])
df_concat

title,text,subject,Flag,id
"“HARRY, HARRY!” C...",President Trump s...,Government News,Fake_News,34359741452
Conservatives Ar...,If it weren t for...,News,Fake_News,8589934724
ERIC TRUMP School...,,left-news,Fake_News,42949675948
MUSLIM TEENS Stag...,"""Oh and as an add...",fjs);}(document,Fake_News,51539608638
Classless Obama C...,Lack of leadershi...,politics,Fake_News,25769803897
Republican Trump'...,WASHINGTON (Reute...,politicsNews,True_News,94489282035
WHO NEEDS DEMOCRA...,No wonder we ve h...,left-news,Fake_News,51539608686
Candace Cameron ...,When you ve never...,News,Fake_News,17179871231
OUCH! WATCH: Snea...,Punishment is no...,left-news,Fake_News,42949676097
10-Year Old Write...,The National Park...,Government News,Fake_News,34359741359


In [12]:
# Remove the title for analysis

df_concat =  df_concat.drop(df_concat["title"])
df_concat

text,subject,Flag,id
President Trump s...,Government News,Fake_News,34359741452
If it weren t for...,News,Fake_News,8589934724
,left-news,Fake_News,42949675948
"""Oh and as an add...",fjs);}(document,Fake_News,51539608638
Lack of leadershi...,politics,Fake_News,25769803897
WASHINGTON (Reute...,politicsNews,True_News,94489282035
No wonder we ve h...,left-news,Fake_News,51539608686
When you ve never...,News,Fake_News,17179871231
Punishment is no...,left-news,Fake_News,42949676097
The National Park...,Government News,Fake_News,34359741359


In [13]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

df_concat

text,subject,Flag,id
president trump s...,Government News,Fake_News,34359741452
if it weren t for...,News,Fake_News,8589934724
,left-news,Fake_News,42949675948
oh and as an add...,fjs);}(document,Fake_News,51539608638
lack of leadershi...,politics,Fake_News,25769803897
washington reute...,politicsNews,True_News,94489282035
no wonder we ve h...,left-news,Fake_News,51539608686
when you ve never...,News,Fake_News,17179871231
punishment is no...,left-news,Fake_News,42949676097
the national park...,Government News,Fake_News,34359741359


In [14]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
df_concat

text,subject,Flag,id,words
president trump s...,Government News,Fake_News,34359741452,"[president, trump..."
if it weren t for...,News,Fake_News,8589934724,"[if, it, weren, t..."
,left-news,Fake_News,42949675948,"[, ]"
oh and as an add...,fjs);}(document,Fake_News,51539608638,"[, oh, and, as, a..."
lack of leadershi...,politics,Fake_News,25769803897,"[lack, of, leader..."
washington reute...,politicsNews,True_News,94489282035,"[washington, , re..."
no wonder we ve h...,left-news,Fake_News,51539608686,"[no, wonder, we, ..."
when you ve never...,News,Fake_News,17179871231,"[when, you, ve, n..."
punishment is no...,left-news,Fake_News,42949676097,"[, punishment, is..."
the national park...,Government News,Fake_News,34359741359,"[the, national, p..."


In [16]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [17]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [18]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [19]:
df_concat

text,subject,Flag,id,words,words without stop
president trump s...,Government News,Fake_News,34359741452,"[president, trump...","[president, trump..."
if it weren t for...,News,Fake_News,8589934724,"[if, it, weren, t...","[electoral, colle..."
,left-news,Fake_News,42949675948,"[, ]","[, ]"
oh and as an add...,fjs);}(document,Fake_News,51539608638,"[, oh, and, as, a...","[, oh, added, bon..."
lack of leadershi...,politics,Fake_News,25769803897,"[lack, of, leader...","[lack, leadership..."
washington reute...,politicsNews,True_News,94489282035,"[washington, , re...","[washington, , re..."
no wonder we ve h...,left-news,Fake_News,51539608686,"[no, wonder, we, ...","[wonder, worst, d..."
when you ve never...,News,Fake_News,17179871231,"[when, you, ve, n...","[never, taught, f..."
punishment is no...,left-news,Fake_News,42949676097,"[, punishment, is...","[, punishment, re..."
the national park...,Government News,Fake_News,34359741359,"[the, national, p...","[national, park, ..."


In [20]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
df_concat

text,subject,Flag,id,words,words without stop,text1
president trump s...,Government News,Fake_News,34359741452,"[president, trump...","[president, trump...",president trump s...
if it weren t for...,News,Fake_News,8589934724,"[if, it, weren, t...","[electoral, colle...",electoral college...
,left-news,Fake_News,42949675948,"[, ]","[, ]",
oh and as an add...,fjs);}(document,Fake_News,51539608638,"[, oh, and, as, a...","[, oh, added, bon...",oh added bonus ...
lack of leadershi...,politics,Fake_News,25769803897,"[lack, of, leader...","[lack, leadership...",lack leadership ...
washington reute...,politicsNews,True_News,94489282035,"[washington, , re...","[washington, , re...",washington reute...
no wonder we ve h...,left-news,Fake_News,51539608686,"[no, wonder, we, ...","[wonder, worst, d...",wonder worst demo...
when you ve never...,News,Fake_News,17179871231,"[when, you, ve, n...","[never, taught, f...",never taught figu...
punishment is no...,left-news,Fake_News,42949676097,"[, punishment, is...","[, punishment, re...",punishment reven...
the national park...,Government News,Fake_News,34359741359,"[the, national, p...","[national, park, ...",national park ser...


In [21]:
df_concat = df_concat.drop("words","words without stop")

df_concat

text,subject,Flag,id,text1
president trump s...,Government News,Fake_News,34359741452,president trump s...
if it weren t for...,News,Fake_News,8589934724,electoral college...
,left-news,Fake_News,42949675948,
oh and as an add...,fjs);}(document,Fake_News,51539608638,oh added bonus ...
lack of leadershi...,politics,Fake_News,25769803897,lack leadership ...
washington reute...,politicsNews,True_News,94489282035,washington reute...
no wonder we ve h...,left-news,Fake_News,51539608686,wonder worst demo...
when you ve never...,News,Fake_News,17179871231,never taught figu...
punishment is no...,left-news,Fake_News,42949676097,punishment reven...
the national park...,Government News,Fake_News,34359741359,national park ser...


In [22]:
df_concat = df_concat.drop("text")

df_concat

subject,Flag,id,text1
Government News,Fake_News,34359741452,president trump s...
News,Fake_News,8589934724,electoral college...
left-news,Fake_News,42949675948,
fjs);}(document,Fake_News,51539608638,oh added bonus ...
politics,Fake_News,25769803897,lack leadership ...
politicsNews,True_News,94489282035,washington reute...
left-news,Fake_News,51539608686,wonder worst demo...
News,Fake_News,17179871231,never taught figu...
left-news,Fake_News,42949676097,punishment reven...
Government News,Fake_News,34359741359,national park ser...


In [23]:
df_concat = df_concat.withColumnRenamed("text1","text")

df_concat

subject,Flag,id,text
Government News,Fake_News,34359741452,president trump s...
News,Fake_News,8589934724,electoral college...
left-news,Fake_News,42949675948,
fjs);}(document,Fake_News,51539608638,oh added bonus ...
politics,Fake_News,25769803897,lack leadership ...
politicsNews,True_News,94489282035,washington reute...
left-news,Fake_News,51539608686,wonder worst demo...
News,Fake_News,17179871231,never taught figu...
left-news,Fake_News,42949676097,punishment reven...
Government News,Fake_News,34359741359,national park ser...


In [24]:
# most frequent words in fake news

# import pyspark.sql.functions as F

# fake_words_freq_df = df_concat.withColumn("text", F.split("text", " ")) \
#     .groupBy("Flag") \
#     .agg(F.collect_list(F.col("text")).alias("text")) \
#     .withColumn("w", F.array_distinct(F.flatten("text"))) \
#     .withColumn("text", F.array_join(F.expr("filter(w, x -> size(filter(text, y -> array_contains(y, x))) = size(text))"), " ")) \
#     .drop("w")

# fake_words_freq_df

In [25]:
from nltk import tokenize

token_space = tokenize.WhitespaceTokenizer()

def counter(text, column_text, quantity):
    all_words = ' '.join([text for text in text[column_text]])
    token_phrase = token_space.tokenize(all_words)
    frequency = nltk.FreqDist(token_phrase)
    df_frequency = pd.DataFrame({"Word": list(frequency.keys()),
                                   "Frequency": list(frequency.values())})
    df_frequency = df_frequency.nlargest(columns = "Frequency", n = quantity)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequency, x = "Word", y = "Frequency", color = 'blue')
    ax.set(ylabel = "Count")
    plt.xticks(rotation='vertical')
    plt.show()

In [40]:
# most frequent words in fake news

df_fake = df_concat.filter(df_concat["Flag"] == "Fake_News")

df_fake_text = df_fake.select("text")

df_fake_text.show()

+--------------------+
|                text|
+--------------------+
|president trump s...|
|electoral college...|
|                    |
| oh added bonus  ...|
|lack leadership  ...|
|wonder worst demo...|
|never taught figu...|
| punishment reven...|
|national park ser...|
|white house publi...|
|sarah palin hear ...|
|https   www youtu...|
|donald trump obse...|
|donald trump hit ...|
|hysterical  sadly...|
|tune alternate cu...|
|hey barack hussei...|
| law order candid...|
|omarosa manigault...|
|one month electio...|
+--------------------+
only showing top 20 rows



In [47]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

+---------+-------+
|     word|  count|
+---------+-------+
|         |1305341|
|    trump|  76773|
|     said|  32978|
|president|  26921|
|   people|  25870|
|      one|  23900|
|    would|  23022|
|  clinton|  18660|
|    obama|  18344|
|     like|  17514|
|   donald|  17160|
|     also|  15029|
|       us|  14363|
|     news|  14179|
|      new|  14095|
|  hillary|  13738|
|     even|  13690|
|     time|  13493|
|    state|  13247|
|    white|  12831|
+---------+-------+
only showing top 20 rows



In [48]:
# most frequent words in true news

df_true = df_concat.filter(df_concat["Flag"] == "True_News")

df_true_text = df_true.select("text")

df_true_text.show()

+--------------------+
|                text|
+--------------------+
|washington  reute...|
|washington  reute...|
|baghdad  reuters ...|
|paris  reuters   ...|
|washington  reute...|
|chicago  reuters ...|
|washington  reute...|
|casper  wyo   reu...|
|washington  reute...|
|washington  reute...|
| reuters    propo...|
|atlanta  reuters ...|
|washington  reute...|
|washington  reute...|
|new york  reuters...|
|moscow  reuters  ...|
|washington  reute...|
|berlin  reuters  ...|
|berlin  reuters  ...|
|washington  reute...|
+--------------------+
only showing top 20 rows



In [49]:
df_true_text = df_true_text\
.withColumn("line", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.withColumn("words", split(col("line"), " "))\
.where(length("line")>0)

df_true_text

text,line,words
washington reute...,washington reute...,"[washington, , re..."
washington reute...,washington reute...,"[washington, , re..."
baghdad reuters ...,baghdad reuters ...,"[baghdad, , reute..."
paris reuters ...,paris reuters ...,"[paris, , reuters..."
washington reute...,washington reute...,"[washington, , re..."
chicago reuters ...,chicago reuters ...,"[chicago, , reute..."
washington reute...,washington reute...,"[washington, , re..."
casper wyo reu...,casper wyo reu...,"[casper, , wyo, ,..."
washington reute...,washington reute...,"[washington, , re..."
washington reute...,washington reute...,"[washington, , re..."
